In [113]:
from selenium import webdriver
from bs4 import BeautifulSoup
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
import pandas as pd
import time

In [114]:
nb_annonce_max = 50
PATH = Service("C:\Program Files (x86)\chromedriver.exe")

In [115]:
url = "https://www.pretapartir.fr/recherche#!/search?th=SJ&sort=promo_desc&depart=A_PAR&f=3_AL&dureeNuit=7-99"

In [116]:
options = webdriver.ChromeOptions()
#options.add_argument('headless')

browser = webdriver.Chrome(service=PATH, options=options)

browser.get(url)

# vu que notre page est dynamique et quelle a besoin normalement de charger pour récupérer les informations et aussi dans le cas ou ik ya eu des mises a jour..
#on dit donc que desque le navigateur ce lance tu attend un certain nombre de temps avant d'executer le code, et dans notre cas ca sera 5s.
time.sleep(5)

#Recuperons le nombres de resultats de la recherche
soup =  BeautifulSoup(browser.page_source, 'html.parser')
nb_elements = int(soup.find('h1').find('strong').get_text(strip=True))


#on defini le nombre de fois que l'on vas devoir cliquer pour charger les pages suivantes
#vue qu'on a récupéré le nombres total de recherche et que nous on veut que lors de l'exécution du programme il vous renvoie maximum 50 resultats de recherche.
nb_click = int(nb_annonce_max/10 - 1)


#appliquer le nombre de click lors du lancement de ma page.
#on dit que quand le navigateur se lance et que le code sexécute vas sur le bouton où on clique pour afficher les autres resultats et click le nombre de fois specifier dans la variable nb_click
for i in range(0, nb_click):
    button_voir_produits_suivant = browser.find_element(By.CSS_SELECTOR, "a[ng-click='vm.loadMorePackage()']")
    
    #on vas exécuté un click en javascript avec selenium (si on utilise click ici il y aura une erreur pcqu'il vas détecté que d'autre éléments détecte le click!)
    browser.execute_script("arguments[0].click()",  button_voir_produits_suivant)
    time.sleep(3) #on vas exécuté des clicks sur le bouton et on dit que sa met 3 secondes chaque fois avant de lancer pour que les informations puisse bien charger et etre récupéré plus facilement
    
    
soup =  BeautifulSoup(browser.page_source, 'html.parser') # on récupére une fois de plus le nouveau contenu de la page pour s'assurrer que les pages des click seront également bien y compris
    
browser.quit()

In [38]:
#Recuperation de tous nos éléments 
items = soup.find_all("div", {"class":"blocProduct-wrapper ng-scope"})
len(items)

50

In [127]:

pays_name_item = (items[0].find("p", {"class":"blocProduct-place"}).get_text(strip=True))

hotel_name_item = items[0].find("p", {"class":"blocProduct-title"}).get_text(strip=True)

detail = items[0].find("div", {"class":"blocProduct-detail"}).findChildren()

duree_sejour_item = detail[0].get_text(strip=True)

date_depart_item = detail[1].get_text()

price = items[0].find("div", {"class":"blocProduct-price"}).findChildren()

price_item= price[1].find("strong", {"class":"ng-binding c-fuschia"}).get_text(strip=True)

print(pays_name_item)
print(hotel_name_item)
print(duree_sejour_item)
print(date_depart_item)
print(price_item)

Grèce
                                - Crète
                                - Héraklion
Grecotel Meli Palace All Inclusive Resort
Séjour de 8 jours / 7 nuits en formule "tout inclus"
Au départ de Paris le 09/05/2023
809€


In [128]:
def clean_text(str):
    return str.replace("\n"," ").replace(" ", "")

items = soup.find_all("div", {"class":"blocProduct-wrapper ng-scope"})

voyages = []

for item in items:
    
    pays = (item.find("p", {"class":"blocProduct-place"}).get_text(strip=True))

    hotel = item.find("p", {"class":"blocProduct-title"}).get_text(strip=True)

    details = item.find("div", {"class":"blocProduct-detail"}).findChildren()

    duree = detail[0].get_text(strip=True)

    date = detail[1].get_text()

    price = item.find("div", {"class":"blocProduct-price"}).findChildren()[1].get_text(strip=True)

    
    
    voyage = {
        "nom du pays":pays,
        "nom de l'hôtel":hotel,
        "duree du séjour":duree,
        "date de depart":date,
        "prix":price,
    }
    
    voyages.append(voyage)
    
voyages

[{'nom du pays': 'Grèce\n                                - Crète\n                                - Héraklion',
  "nom de l'hôtel": 'Grecotel Meli Palace All Inclusive Resort',
  'duree du séjour': 'Séjour de 8 jours / 7 nuits en formule "tout inclus"',
  'date de depart': 'Au départ de Paris le 09/05/2023',
  'prix': '809€/ pers'},
 {'nom du pays': 'Tunisie\n                                \n                                - Tunis',
  "nom de l'hôtel": 'Occidental Sousse Marhaba',
  'duree du séjour': 'Séjour de 8 jours / 7 nuits en formule "tout inclus"',
  'date de depart': 'Au départ de Paris le 09/05/2023',
  'prix': '1627€/ pers'},
 {'nom du pays': 'Maldives\n                                \n                                - Malé',
  "nom de l'hôtel": 'Adaaran Select Meedhupparu Resort',
  'duree du séjour': 'Séjour de 8 jours / 7 nuits en formule "tout inclus"',
  'date de depart': 'Au départ de Paris le 09/05/2023',
  'prix': '2234€/ pers'},
 {'nom du pays': 'Grèce\n          

In [129]:
df = pd.DataFrame(voyages)
df

,nom du pays,nom de l'hôtel,duree du séjour,date de depart,prix
0,Grèce\n - Crète...,Grecotel Meli Palace All Inclusive Resort,"Séjour de 8 jours / 7 nuits en formule ""tout i...",Au départ de Paris le 09/05/2023,809€/ pers
1,Tunisie\n \n ...,Occidental Sousse Marhaba,"Séjour de 8 jours / 7 nuits en formule ""tout i...",Au départ de Paris le 09/05/2023,1627€/ pers
2,Maldives\n \n ...,Adaaran Select Meedhupparu Resort,"Séjour de 8 jours / 7 nuits en formule ""tout i...",Au départ de Paris le 09/05/2023,2234€/ pers
3,Grèce\n \n ...,Atrium Palace Thalasso Spa Resort & Villas,"Séjour de 8 jours / 7 nuits en formule ""tout i...",Au départ de Paris le 09/05/2023,1427€/ pers
4,Tunisie\n \n ...,Hôtel Sentido Djerba Beach,"Séjour de 8 jours / 7 nuits en formule ""tout i...",Au départ de Paris le 09/05/2023,634€/ pers
5,Espagne\n - Ile...,Hôtel H10 Playa Meloneras,"Séjour de 8 jours / 7 nuits en formule ""tout i...",Au départ de Paris le 09/05/2023,1219€/ pers
6,Italie\n - Sici...,Club Lipari,"Séjour de 8 jours / 7 nuits en formule ""tout i...",Au départ de Paris le 09/05/2023,679€/ pers
7,Grèce\n \n ...,White Olive Premium Lindos,"Séjour de 8 jours / 7 nuits en formule ""tout i...",Au départ de Paris le 09/05/2023,690€/ pers
8,Sénégal\n \n ...,Le Saly,"Séjour de 8 jours / 7 nuits en formule ""tout i...",Au départ de Paris le 09/05/2023,1095€/ pers
9,Maldives\n \n ...,Sun Island Resort & Spa,"Séjour de 8 jours / 7 nuits en formule ""tout i...",Au départ de Paris le 09/05/2023,1877€/ pers


In [132]:
df.to_excel("scraping-3.xlsx", index=False, encoding = 'UTF-8')